<a href="https://colab.research.google.com/github/vijaykumarharakuni/mllearning/blob/main/Recommoder_Suprise_package.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505185 sha256=fe8b01058f2cc5b82af125b75ab6df862a06af1382dcc90814f590f28fc029c1
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [2]:
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy

In [14]:
# Step 1: Define the dataset
# Example: user_id, item_id, rating
data_dict = {
    "user_id": ["U1", "U1", "U1", "U2", "U2", "U3", "U3"],
    "item_id": ["I1", "I2", "I3", "I1", "I4", "I3", "I4"],
    "rating": [4, 5, 3, 5, 2, 5, 4],
}

In [15]:
# Step 2: Load data into Surprise format
reader = Reader(rating_scale=(1, 5))  # Ratings range between 1 and 5
data = Dataset.load_from_df(
    pd.DataFrame(data_dict), reader
)

In [5]:
# Step 3: Train-Test Split
trainset, testset = train_test_split(data, test_size=0.2)

In [16]:
# Step 4: Define the KNNBasic algorithm (user-based filtering)
sim_options = {
    "name": "msd",  # Use cosine similarity , cosine, pearson,pearson_baseline
    "user_based": True,  # User-based filtering
}
algo = KNNBasic(sim_options=sim_options)

# Step 5: Train the model
algo.fit(trainset)

# Step 6: Test the model and predict
predictions = algo.test(testset)
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8602


0.8602325267042628

In [17]:
# Step 7: Predict a specific rating
uid, iid = "U1", "I4"  # User 1, Item 4
pred = algo.predict(uid, iid)
print(f"Predicted rating for {uid} on {iid}: {pred.est:.2f}")

Predicted rating for U1 on I4: 4.00


In [18]:
# content based similarity
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Item features matrix (items x features)
# Features could be, for example: [Action, Comedy, Drama, Sci-Fi]

item_features = np.array([
    [1, 0, 0, 1],  # Item 1 (Action, Sci-Fi)
    [0, 1, 0, 1],  # Item 2 (Comedy, Sci-Fi)
    [1, 0, 1, 0],  # Item 3 (Action, Drama)
    [0, 1, 1, 0],  # Item 4 (Comedy, Drama)
])

In [19]:
item_features

array([[1, 0, 0, 1],
       [0, 1, 0, 1],
       [1, 0, 1, 0],
       [0, 1, 1, 0]])

In [20]:
# User's preferences (based on past ratings or likes)
# For example, the user likes Item 1 and Item 3 (indicating that they prefer Action movies)
user_profile = np.array([1, 0, 1, 0])  # User likes Action and Drama genres

# Calculate the cosine similarity between items
item_similarity = cosine_similarity(item_features)

# Recommend items for the user based on the highest similarity to their profile
# Calculate similarity between user profile and each item
user_item_similarity = cosine_similarity(user_profile.reshape(1, -1), item_features).flatten()

# Get the index of items sorted by similarity to user profile (in descending order)
recommended_items = np.argsort(user_item_similarity)[::-1]

# Print the recommended items
print("Recommended Items for the User:")
for item_idx in recommended_items:
    print(f"Item {item_idx + 1} (Similarity Score: {user_item_similarity[item_idx]:.2f})")

Recommended Items for the User:
Item 3 (Similarity Score: 1.00)
Item 4 (Similarity Score: 0.50)
Item 1 (Similarity Score: 0.50)
Item 2 (Similarity Score: 0.00)
